In [141]:
import pickle
import os
import ast
import json
import pandas as pd
from tqdm import tqdm
from collections import Counter, defaultdict
from json_repair import repair_json
# good_json_string = repair_json(bad_json_string)

In [140]:
output_path = '../data/outputs/qwen-3-235b-a22b-instruct-2507/'

In [143]:
dfcomp = pd.read_csv('../code/comps_props.csv').dropna().reset_index(drop=True)
compounds = list(dfcomp.name.values)

In [11]:
import json

with open('../data/outputs/qwen-3-235b-a22b-instruct-2507/CN101033114A.json', 'r') as f:
    data = json.load(f)

In [147]:
js2excel = {
 'CN101033114A.json': 'CN101033114 _table-33959.xlsx',
 'CN101058477A.json': 'CN101058477_table-35472.xlsx',
 'CN101072676A.json': 'CN101072676A_table-30659.xlsx',
 'CN101092308B.json': 'CN101092308B_table-35489.xlsx',
 'CN1308591A.json': 'CN1308591_table-24104.xlsx',
 'CN1326903A.json': 'CN1326903_table-28556.xlsx',
 'CN1489965A.json': 'CN1489965_table-37874.xlsx',
 'CN1495138A.json': 'CN1495138_table-30623.xlsx',
 'CN1522980A.json': 'CN1522980_table-28312.xlsx',
 'CN1771211A.json': 'CN1771211_table-28366.xlsx',
 'FR_1027316_A.json': 'FR1027316_table-16101.xlsx',
 'FR_1172977_A.json': 'FR1172977_table-29832.xlsx',
 'FR_1242292_A.json': 'FR1242292_table-37871.xlsx',
 'FR_1247560_A.json': 'FR1247560_table-29840.xlsx',
 'FR_1388413_A.json': 'FR1388413_table-17489.xlsx',
 'FR_1484178_A.json': 'FR1484178_table-30674.xlsx',
 'FR_2511360_A1.json': 'FR2511360_table-8590.xlsx',
 'FR_2564087_A1.json': 'FR2564087_table-8498.xlsx',
 'FR_2650266_A1.json': 'FR2650266_table-4410.xlsx',
 'FR_2775476_A1.json': 'FR2775476_table-17437.xlsx',
 'JP2013209231A.json': 'JP2013209231_table-41689.xlsx',
 'JP2013520387A.json': 'JP2013520387_table-39169.xlsx',
 'JP2013520388A.json': 'JP2013520388_table-43286.xlsx',
 'JP2014019632A.json': 'JP2014019632_table-41679.xlsx',
 'JP2014504250A.json': 'JP2014504250_table-39897.xlsx',
 'JP2015013773A.json': 'JP2015013773_table-42776.xlsx',
 'JP2015509903A.json': 'JP2015509903_table-43872.xlsx',
 'JP5454585B2.json': 'JP5454585_table-40257.xlsx',
 'JP5613164B2.json': 'JP5613164_table-40102.xlsx',
 'JP5743125B2.json': 'JP5743125_table-43288.xlsx',
 'JP5763852B2.json': 'JP5763852_table-44883.xlsx',
 'KR20160040680A.json': 'KR20160040680_table-42977.xlsx',
 'KR20170007352A.json': 'KR20170007352_table-43529.xlsx',
 'KR20170021584A.json': 'KR20170021584_table-43757.xlsx',
    'RU2016861C1.json': 'RU2016861_table-4306.xlsx',
    'RU2017695C1.json': 'RU2017695_table-4488.xlsx',
    'RU2045486C1.json': 'RU2045486_table-15262.xlsx',
    'RU2069198C1.json': 'RU2069198_table-15265.xlsx',
    'RU2102345C1.json': 'RU2102345_table-2783.xlsx',
    'RU_2015120_C1.json':'RU2015120_table-15217.xlsx',
    'RU_2016858_C1.json': 'RU2016858_table-4323.xlsx',
    'RU_2021219_C1.json': 'RU2021219_table-4489.xlsx',
    'RU_2044710_C1.json': 'RU2044710_table-8002.xlsx',
    'RU_2383502_C1.json': 'RU2383502_table-37098.xlsx',
 'US1304623.json': 'US1304623_table-24764.xlsx',
 'US1623301.json': 'US1623301_table-24156.xlsx',
 'US20010014424A1.json': 'US20010014424A1_table-33582.xlsx',
 'US20030228968A1.json': 'US20030228968A1_table-28530.xlsx',
 'US20040214047A1.json': 'US20040214047A1_table-28846.xlsx',
 'US20060166804A1.json': 'US20060166804A1_table-33051.xlsx',
 'US20090239122A1.json': 'US2009239122A1_table-30659.xlsx',
 'US2486812.json':'US2486812_table-16059.xlsx',
 'US2920971.json':'US2920971_table-37870.xlsx',     
  
}

excel2js = dict()

for k, v in js2excel.items():
    
    excel2js[v] = k
    
    # 'US3640738_table-13146.xlsx', 'US20090286667A1.json': ,

In [248]:

js = 'CN101033114A.json'

noex = []

for js in js2excel.keys():
    try:
        with open(f'../data/outputs/qwen-3-235b-a22b-instruct-2507/{js}', 'r') as f:
            data = json.load(f)['answer']
        
        
        column_dict = dict()
        
        files = os.listdir('../data/excel/')
        files.remove('.DS_Store')
        
        mapper = dict()
        ex = js2excel[js]
        
        df = pd.read_excel(f'../data/excel/{ex}')
        columns = df.columns.tolist()
        idx = df[df[columns[0]] == 'Glass No'].index[-1]
        
        idx0 = df[df[columns[0]] == 'Glass No'].index[0]
        idx1 = df[df[columns[0]] == 'Glass No'].index[-1]
        
        for k,v in zip(df.iloc[idx0], df.iloc[idx1]):
            k = k.replace("\n", ' ')
            if k in mapper:
                mapper[k].append(v)
            else:
                mapper[k] = [v]
        dff = df.iloc[idx:,:]
        dff.columns = df.iloc[idx]
        dff = dff.iloc[1:,:]
        column_dict[file] = dff.columns.tolist()
        
        gold = []
        for i in range(len(dff)):
            gold.append(dff.iloc[i].to_dict())
        
        
        gold_comps = []
        for g in gold:
            compd = dict()
            for k,v in g.items():
                if k in compounds:
                    try:
                        num = float(g[k])
                        compd[k] = v
                    except:
                        pass
            gold_comps.append(compd)
        
        tups = []
        
        for gold_comp in gold_comps:
            idx = 0
            for k,v in gold_comp.items():
                tups.append((f'{ex}_{idx}', k, v, 'unit'))
                idx+=1
        
        pred_comps = []
        
        for g in ast.literal_eval(data):
        
            compd = dict()
            for k,v in g[0].items():
                if k in compounds:
                    try:
                        num = float(g[0][k])
                        compd[k] = v
                    except:
                        pass
            pred_comps.append(compd)
        
        pred_tups = []
        
        for pred_comp in pred_comps:
            idx = 0
            for k,v in pred_comp.items():
                pred_tups.append((f'{ex}_{idx}', k, v, 'unit'))
                idx+=1
        
        # precision
        cnt = 0
        for d in gold_comps:
            if d in pred_comps:
                cnt += 1
        
        # recall
        cnt2 = 0
        for d in pred_comps:
            if d in gold_comps:
                cnt2 += 1
        
        print(cnt, cnt2, len(gold_comps), len(pred_comps))
    except:
        noex.append(ff)

6 6 8 6
4 4 4 4
6 6 25 6
17 17 23 22
8 8 8 68
0 0 15 14
5 9 5 14
2 2 6 7
0 0 5 5
0 0 1 2
0 0 4 4
5 5 10 18
3 3 3 3
3 3 32 42
23 25 23 40
13 13 15 15
0 0 6 6
0 0 5 13
2 2 7 7
3 3 4 4
1 1 3 2
0 0 5 9
0 0 6 4
5 5 29 29
0 0 8 2
29 29 68 30
21 21 26 26
0 0 12 12
9 9 12 10
3 3 4 4
5 5 5 5
8 8 9 10
5 5 5 6
4 4 11 11
3 3 3 3
6 6 14 15
5 5 5 5
3 3 3 3
5 5 6 6
3 4 9 8
0 0 68 13
3 3 5 8
6 6 25 6
0 0 5 31
8 8 51 88


In [240]:
# dff[dff['SiO2'] == 57.6]

(6, 6, 8, 6)

In [236]:
17/23

0.7391304347826086

In [233]:
pred_comps

[{'ZnO': 59.2, 'B2O3': 20.8, 'SiO2': 17.9, 'Al2O3': 2.1},
 {'ZnO': 64.5, 'B2O3': 20.8, 'SiO2': 12.7, 'Al2O3': 2.0},
 {'ZnO': 59.1, 'B2O3': 26.6, 'SiO2': 13.1, 'Al2O3': 1.2},
 {'ZnO': 53.4, 'B2O3': 26.4, 'SiO2': 13.9, 'Bi2O3': 5.0, 'Al2O3': 1.3},
 {'ZnO': 48.8, 'B2O3': 26.3, 'SiO2': 14.2, 'Bi2O3': 10.0, 'Al2O3': 0.7},
 {'ZnO': 55.4, 'B2O3': 26.6, 'SiO2': 12.6, 'CuO': 4.0, 'Al2O3': 1.4},
 {'ZnO': 49.1, 'B2O3': 26.5, 'SiO2': 12.8, 'CuO': 10.1, 'Al2O3': 1.5},
 {'ZnO': 44.2, 'B2O3': 26.6, 'SiO2': 12.8, 'CuO': 15.0, 'Al2O3': 1.4},
 {'ZnO': 39.5, 'B2O3': 26.5, 'SiO2': 12.7, 'CuO': 19.8, 'Al2O3': 1.5},
 {'ZnO': 44.3, 'B2O3': 26.4, 'SiO2': 15.3, 'CuO': 9.2, 'Bi2O3': 4.8},
 {'ZnO': 44.0,
  'B2O3': 26.6,
  'SiO2': 12.8,
  'CuO': 10.0,
  'Bi2O3': 5.0,
  'Al2O3': 1.6,
  'CoO': 1.8},
 {'ZnO': 42.2,
  'B2O3': 26.4,
  'SiO2': 13.3,
  'CuO': 9.0,
  'Bi2O3': 5.0,
  'Al2O3': 2.3},
 {'ZnO': 39.4,
  'B2O3': 24.8,
  'SiO2': 12.7,
  'CuO': 9.0,
  'Bi2O3': 4.7,
  'Al2O3': 1.9,
  'Fe2O3': 1.8},
 {'ZnO': 39.3,


In [216]:
def sort_comp(single_list_comp):
    '''
    for sorting  ordering of chemical components alphabetically
    '''
    single_list_comp_sorted = sorted(single_list_comp, key=lambda x: x[0])
    return single_list_comp_sorted

def get_composition_metrics_without_ids(gold_tuples, pred_tuples):
    gold_comps, pred_comps = defaultdict(set), defaultdict(set)
    
    for g in gold_tuples:
        underscore_count = g[0].count('_')
        if underscore_count == 5:
            new_item = '_'.join(g[0].split('_')[:-1])
        else:
            new_item = g[0]
        gold_comps[new_item].add((g[1], round(g[2], 2), g[3]))
        # print(gold_comps[g[0]])

    for p in pred_tuples:
        underscore_count = p[0].count('_')
        if underscore_count == 5:
            new_item = '_'.join(p[0].split('_')[:-1])
        else:
            new_item = p[0]
        pred_comps[new_item].add((p[1], round(p[2], 2), p[3]))
        
   
    for p, v in pred_comps.items():
        pred_comps[p] = set(sort_comp(list(v)))
        
    for g, v in gold_comps.items():
        gold_comps[g] = set(sort_comp(list(v)))

    prec = 0
    for p, v in pred_comps.items():
        if p in gold_comps and gold_comps[p] == v:
            prec += 1
    if len(pred_comps) > 0:
        prec /= len(pred_comps)
    else:
        prec = 0.0
    rec = 0
    for g, v in gold_comps.items():
        if g in pred_comps and pred_comps[g] == v:
            rec += 1
    rec /= len(gold_comps)
    fscore = 2 * prec * rec / (prec + rec) if (prec + rec > 0) else 0.0
    metrics = {'precision': prec, 'recall': rec, 'fscore': fscore}
    metrics = {m: round(v * 100, 2) for m, v in metrics.items()}
    return metrics

In [217]:
get_composition_metrics_without_ids(tups, pred_tups)

{'precision': 28.57, 'recall': 28.57, 'fscore': 28.57}

In [223]:
demogold = [(t[1], t[2]) for t in tups]
demotest = [(t[1], t[2]) for t in pred_tups]

cnt = 0
for d in demogold:
    if d in demotest:
        cnt += 1

cnt2 = 0
for d in demotest:
    if d in demogold:
        cnt2 += 1

cnt, cnt2, len(demogold)

In [230]:
cnt = 0
for d in demogold:
    if d in demotest:
        cnt += 1

cnt2 = 0
for d in demotest:
    if d in demogold:
        cnt2 += 1

In [231]:
cnt, cnt2, len(demogold)

(127, 124, 131)

In [232]:
124/131

0.9465648854961832

In [219]:
pred_tups

[('CN101092308B_table-35489.xlsx_0', 'ZnO', 59.2, 'unit'),
 ('CN101092308B_table-35489.xlsx_1', 'B2O3', 20.8, 'unit'),
 ('CN101092308B_table-35489.xlsx_2', 'SiO2', 17.9, 'unit'),
 ('CN101092308B_table-35489.xlsx_3', 'Al2O3', 2.1, 'unit'),
 ('CN101092308B_table-35489.xlsx_0', 'ZnO', 64.5, 'unit'),
 ('CN101092308B_table-35489.xlsx_1', 'B2O3', 20.8, 'unit'),
 ('CN101092308B_table-35489.xlsx_2', 'SiO2', 12.7, 'unit'),
 ('CN101092308B_table-35489.xlsx_3', 'Al2O3', 2.0, 'unit'),
 ('CN101092308B_table-35489.xlsx_0', 'ZnO', 59.1, 'unit'),
 ('CN101092308B_table-35489.xlsx_1', 'B2O3', 26.6, 'unit'),
 ('CN101092308B_table-35489.xlsx_2', 'SiO2', 13.1, 'unit'),
 ('CN101092308B_table-35489.xlsx_3', 'Al2O3', 1.2, 'unit'),
 ('CN101092308B_table-35489.xlsx_0', 'ZnO', 53.4, 'unit'),
 ('CN101092308B_table-35489.xlsx_1', 'B2O3', 26.4, 'unit'),
 ('CN101092308B_table-35489.xlsx_2', 'SiO2', 13.9, 'unit'),
 ('CN101092308B_table-35489.xlsx_3', 'Bi2O3', 5.0, 'unit'),
 ('CN101092308B_table-35489.xlsx_4', 'Al2O3'

In [218]:
tups

[('CN101092308B_table-35489.xlsx_0', 'ZnO', 59.2, 'unit'),
 ('CN101092308B_table-35489.xlsx_1', 'B2O3', 20.8, 'unit'),
 ('CN101092308B_table-35489.xlsx_2', 'SiO2', 17.9, 'unit'),
 ('CN101092308B_table-35489.xlsx_3', 'Al2O3', 2.1, 'unit'),
 ('CN101092308B_table-35489.xlsx_0', 'ZnO', 64.5, 'unit'),
 ('CN101092308B_table-35489.xlsx_1', 'B2O3', 20.8, 'unit'),
 ('CN101092308B_table-35489.xlsx_2', 'SiO2', 12.7, 'unit'),
 ('CN101092308B_table-35489.xlsx_3', 'Al2O3', 2, 'unit'),
 ('CN101092308B_table-35489.xlsx_0', 'ZnO', 59.1, 'unit'),
 ('CN101092308B_table-35489.xlsx_1', 'B2O3', 26.6, 'unit'),
 ('CN101092308B_table-35489.xlsx_2', 'SiO2', 13.1, 'unit'),
 ('CN101092308B_table-35489.xlsx_3', 'Al2O3', 1.2, 'unit'),
 ('CN101092308B_table-35489.xlsx_0', 'ZnO', 53.4, 'unit'),
 ('CN101092308B_table-35489.xlsx_1', 'B2O3', 26.4, 'unit'),
 ('CN101092308B_table-35489.xlsx_2', 'SiO2', 13.9, 'unit'),
 ('CN101092308B_table-35489.xlsx_3', 'Bi2O3', 5, 'unit'),
 ('CN101092308B_table-35489.xlsx_4', 'Al2O3', 1.

In [155]:
print(get_composition_metrics_without_ids([('CN101033114 _table-33959.xlsx_0', 'SiO2', 73, 'unit'),
 ('CN101033114 _table-33959.xlsx_1', 'Al2O3', 1, 'unit'),
 ('CN101033114 _table-33959.xlsx_2', 'B2O3', 22, 'unit')], [('CN101033114 _table-33959.xlsx_0', 'SiO2', 73, 'unit'),
 ('CN101033114 _table-33959.xlsx_1', 'Al2O3', 1, 'unit'),
 ('CN101033114 _table-33959.xlsx_2', 'B2O3', 22, 'unit')]))

{('SiO2', 73, 'unit')}
{('Al2O3', 1, 'unit')}
{('B2O3', 22, 'unit')}
{'precision': 100.0, 'recall': 100.0, 'fscore': 100.0}


In [52]:
outputs = []
for ff in tqdm(os.listdir(output_path)):
    with open(f'../data/outputs/qwen-3-235b-a22b-instruct-2507/{ff}', 'r') as f:
        data = json.load(f)['answer']
        outputs.append(data)
        

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 55/55 [00:00<00:00, 3833.28it/s]


In [58]:
column_dict = dict()

files = os.listdir('../data/excel/')
files.remove('.DS_Store')

mapper = dict()

for file in :
    df = pd.read_excel(f'../data/excel/{file}')
    columns = df.columns.tolist()
    idx = df[df[columns[0]] == 'Glass No'].index[-1]
    
    idx0 = df[df[columns[0]] == 'Glass No'].index[0]
    idx1 = df[df[columns[0]] == 'Glass No'].index[-1]

    for k,v in zip(df.iloc[idx0], df.iloc[idx1]):
        k = k.replace("\n", ' ')
        if k in mapper:
            mapper[k].append(v)
        else:
            mapper[k] = [v]
    dff = df.iloc[idx:,:]
    dff.columns = df.iloc[idx]
    dff = dff.iloc[1:,:]
    column_dict[file] = dff.columns.tolist()

In [57]:
ast.literal_eval(outputs[5])#:7]

[[{'SiO2': 70.619,
   'Al2O3': 6.0,
   'Na2O': 17.0,
   'CaO': 2.0,
   'MgO': 0.01,
   'F': 4.0,
   'P2O5': 0.1,
   'Fe2O3': 0.1,
   'SO3': 0.1,
   'Co2O3': 0.005,
   'CuO': 0.005,
   'Cr2O3': 0.001,
   'MnO': 0.005,
   'NiO': 0.05,
   'Se': 0.005},
  {'softening_temperature': [{'value': 620,
     'unit': '°C',
     'experimental_conditions': ''}]}],
 [{'SiO2': 62.64,
   'Al2O3': 3.0,
   'Na2O': 15.0,
   'CaO': 12.0,
   'MgO': 5.0,
   'F': 1.0,
   'P2O5': 1.0,
   'Fe2O3': 0.01,
   'SO3': 0.3,
   'Co2O3': 0.05},
  {'softening_temperature': [{'value': 630,
     'unit': '°C',
     'experimental_conditions': ''}]}],
 [{'SiO2': 60.0,
   'Al2O3': 7.0,
   'Na2O': 10.0,
   'CaO': 12.0,
   'MgO': 3.55,
   'F': 4.0,
   'P2O5': 2.0,
   'Fe2O3': 0.5,
   'SO3': 0.7,
   'NiO': 0.25},
  {'softening_temperature': [{'value': 650,
     'unit': '°C',
     'experimental_conditions': ''}]}],
 [{'SiO2': 65.65,
   'Al2O3': 5.0,
   'Na2O': 12.0,
   'CaO': 6.0,
   'MgO': 4.0,
   'F': 3.0,
   'P2O5': 3.0,
   'F

In [24]:
jsons = os.listdir(output_path)
excels = os.listdir('../data/excel/')
excels.remove('.DS_Store')

In [31]:
js2excel = dict()
excel2js = dict()

for js, ex in zip(sorted(jsons), sorted(excels)):
    js2excel[js] = ex
    excel2js[ex] = js